In [2]:
from zenml import step
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

@step
def local_data_loading() -> dict:
    """Load the iris dataset from scikit-learn."""
    iris = load_iris()
    X = iris.data
    y = iris.target
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    return {'X_train': X_train, 'X_test': X_test, 'y_train': y_train, 'y_test': y_test}


In [3]:
from zenml import step
from sklearn.linear_model import LogisticRegression

@step(step_operator="k8s")
def remote_model_training(data: dict) -> LogisticRegression:
    """Train a logistic regression model on the iris dataset."""
    model = LogisticRegression()
    model.fit(data['X_train'], data['y_train'])
    
    train_accuracy = model.score(data['X_train'], data['y_train'])
    test_accuracy = model.score(data['X_test'], data['y_test'])
    
    print(f"Train accuracy: {train_accuracy:.2f}")
    print(f"Test accuracy: {test_accuracy:.2f}")
    
    return model

In [ ]:
from zenml import pipeline

@pipeline
def local_and_remote_pipeline():
    """Define a pipeline that connects the steps."""
    data = local_data_loading()
    model = remote_model_training(data)

1
Initiating a new run for the pipeline: simple-demo-notebook.
Registered new version: (version 1).
Unable to find a build to reuse. When using a code repository, a previous build can be reused when the following conditions are met:
  * The existing build was created for the same stack, ZenML version and Python version
  * The stack contains a container registry
  * The Docker settings of the pipeline and all its steps are the same as for the existing build
  * The build does not include code. This will only be the case if the existing build was created with a clean code repository.
Building Docker image(s) for pipeline simple-demo-notebook.
Building Docker image us-central1-docker.pkg.dev/shopify-ml-offline-sandbox/zenml-785362/zenml:simple-demo-notebook-remote_step-kubernetes_step_operator.
- Including stack requirements: gcsfs, google-cloud-aiplatform>=1.34.0, google-cloud-artifact-registry>=1.11.3, google-cloud-build>=3.11.0, google-cloud-container>=2.21.0, google-cloud-secret-mana

I0000 00:00:1721861615.630586  826368 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache
I0000 00:00:1721861615.641420  826368 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


Running Cloud Build to build the Docker image. Cloud Build logs: https://console.cloud.google.com/cloud-build/builds/29f9ccc2-b7c8-434e-8611-f9990f39525e?project=405564901221


In [ ]:
run = local_and_remote_pipeline()
print("success")

In [ ]:
print(run.steps["remote_model_training"].output.load())